<a href="https://colab.research.google.com/github/HoangShiya/HUET/blob/main/THI_H%E1%BB%8CC_PH%E1%BA%A6N_PH%C3%82N_T%C3%8DCH_ANOVA_1_CHI%E1%BB%80U.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **PHÂN TÍCH ANOVA 1 CHIỀU VỚI TẬP DỮ LIỆU "Diet_R.csv"**
---
---

Sử dụng các thư viện cần thiết

In [2]:
from google.colab import drive
import pandas as pd
import statistics as sts
import numpy as np 
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

> **Bước 1**: Tiến hành lấy tập dữ liệu owan01.csv từ drive để tiến hành phân tích

In [3]:
drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Python/Diet_R.csv')
print(f"Dữ liệu của tập dữ liệu Diet_R.csv:\n ----------------------------------\n {df}")

Mounted at /content/drive
Dữ liệu của tập dữ liệu Diet_R.csv:
 ----------------------------------
     Person  gender  Age  Height  pre.weight  Diet  weight6weeks
0       25     NaN   41     171          60     2          60.0
1       26     NaN   32     174         103     2         103.0
2        1     0.0   22     159          58     1          54.2
3        2     0.0   46     192          60     1          54.0
4        3     0.0   55     170          64     1          63.3
..     ...     ...  ...     ...         ...   ...           ...
73      74     1.0   35     183          83     3          80.2
74      75     1.0   49     177          84     3          79.9
75      76     1.0   28     164          85     3          79.7
76      77     1.0   40     167          87     3          77.8
77      78     1.0   51     175          88     3          81.9

[78 rows x 7 columns]


Chuyển đổi df thành dạng phù hợp để phân tích ANOVA một chiều theo gói scipy.stats, trường hợp này có thể chuyển dữ liệu sang dạng list và  sử dụng hàm *dropna()*

> **Cú pháp**: dataFrame.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

*Chú thích*:

1) *axis*: 0 là "index", 1 là "column", mặc định axis = 0

Xác định xem các hàng hoặc cột chứa các giá trị bị thiếu có bị xóa hay không.

*   0, hoặc "index": Thả các hàng chứa các giá trị bị thiếu.
*   1 hoặc "column": Bỏ các cột chứa giá trị bị thiếu.

2) *how*: "any" hoặc "all", mặc định là any

Xác định xem hàng hoặc cột có bị xóa khỏi DataFrame hay không khi chúng ta có ít nhất một NA hoặc tất cả NA.

*   "any": Nếu có bất kỳ giá trị NA nào sẽ loại bỏ luôn hàng hoặc cột đó.
*   "all": Nếu tất cả các giá trị đều là NA sẽ loại luôn bỏ hàng hoặc cột đó.

3) *thresh*: int, optional

Yêu cầu nhiều giá trị không phải NA

4) *subset*: Nhãn cột (column label), chuỗi nhãn (sequence label) hoặc tùy chọn (optional)

Xem xét các nhãn dọc theo trục khác nhau.

Ví dụ: Nếu bạn đang bỏ hàng thì đây sẽ là danh sách bao gồm các cột cần thiết.

5) *inplace*: bool, mặc định là "sai" (False)

Nếu Đúng, hãy thực hiện thao tác tại chỗ và trả về giá trị None.

> **Chức năng**: Loại bỏ những giá trị bị thiếu

> **Giá trị trả về**: Dataframe hoặc None

DataFrame với các mục NA bị loại bỏ khỏi nó hoặc Không có nếu inplace=True.


In [4]:
# Khởi tạo giá trị alpha = 0.05 là ý nghĩa của quá trình kiểm định
alpha = 0.05

# Khởi tạo biến sample_converted_by_df dùng để lưu trữ dữ liệu của biến df khi được chuyển đổi sang dạng list và sàng lọc dữ liệu
sample_converted_by_df = [list(df["Person"].dropna()), 
                          list(df["gender"].dropna()),
                          list(df["Age"].dropna()),
                          list(df["Height"].dropna()),
                          list(df["pre.weight"].dropna()),
                          list(df["Diet"].dropna()),
                          list(df["weight6weeks"].dropna())]

print(f"Dữ liệu của tập dữ liệu owan01.csv khi được chuyển đổi và xử lí số liệu:\n --------------------------------------------------------------------\n {sample_converted_by_df}")

Dữ liệu của tập dữ liệu owan01.csv khi được chuyển đổi và xử lí số liệu:
 --------------------------------------------------------------------
 [[25, 26, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [41, 32, 22, 46, 55, 33, 50, 50, 37, 28, 28, 45, 60, 48, 41, 37, 44, 37, 41, 43, 20, 51, 31, 54, 50, 48, 16, 37, 30, 29, 51, 35, 21, 22, 36, 20, 35, 45, 58, 37, 31, 35,

### a) Kiểm định các nhóm có tuân theo phân phối chuẩn không

> **Bước 2.a**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Nhóm tuân theo phân phối chuẩn}$$

Giả thuyết $H_a$:

$$H_a: \text{Nhóm không tuân theo phân phối chuẩn}$$

> **Bước 3.a**: Tiến hành kiểm định và tính toán các đại lượng cần thiết

> **Bước 4.a**: Kết luận dựa trên kết quả kiểm định

In [5]:
# Để kiểm định từng hàng, ta sử dụng vòng lặp for và so sánh giá trị p với mức ý nghĩa để tiến hành kiểm định
for i in range(0, 7, 1):
  p = st.shapiro(sample_converted_by_df[i])[1]

  # Phát biểu kết quả kiểm định từng nhóm
  print("-------------------------------------------------------------------------")
  print(f"Giá trị p của nhóm thứ {i+1} là: {np.round(p, 2)}")
  if (p < alpha):
    print(f"Vì giá trị p của nhóm thứ {i+1} bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
  else:
    print(f"Vì giá trị p của nhóm thứ {i+1} không bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

print("-------------------------------------------------------------------------")

-------------------------------------------------------------------------
Giá trị p của nhóm thứ 1 là: 0.01
Vì giá trị p của nhóm thứ 1 bé hơn mức ý nghĩa của bài toán (0.01 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 2 là: 0.0
Vì giá trị p của nhóm thứ 2 bé hơn mức ý nghĩa của bài toán (0.0 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 3 là: 0.65
Vì giá trị p của nhóm thứ 3 không bé hơn mức ý nghĩa của bài toán (0.65 >= 0.05). Vì vậy có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá trị p của nhóm thứ 4 là: 0.0
Vì giá trị p của nhóm thứ 4 bé hơn mức ý nghĩa của bài toán (0.0 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0
-------------------------------------------------------------------------
Giá t

### b) Kiểm định sự đồng nhất giữa phương sai các nhóm

> **Bước 2.b**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về phương sai của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về phương sai của các nhóm}$$

> **Bước 3.b**: Tiến hành kiểm định và tính toán các đại lượng cần thiết


In [8]:
# Khởi tạo biến stt, p để lưu giá trị của stat value và p value
stt, p = st.levene(sample_converted_by_df[0], sample_converted_by_df[1], sample_converted_by_df[2], sample_converted_by_df[3], sample_converted_by_df[4], sample_converted_by_df[5], sample_converted_by_df[6])   

# Xuất kết quả stat value, p value ra màn hình
print(f"Giá trị stat: {np.round(stt, 2)}")
print(f"Giá trị p: {np.round(p, 2)}")

Giá trị stat: 77.96
Giá trị p: 0.0


> **Bước 4.b**: Kết luận dựa trên kết quả kiểm định

In [9]:
if (p < alpha):
  print(f"Vì giá trị p  bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p không bé hơn mức ý nghĩa của bài toán ({np.round(p, 2)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p  bé hơn mức ý nghĩa của bài toán (0.0 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0


Vì các nhóm độc lập, tuân theo phân phối chuẩn và có sự đồng nhất về phương sai giữa các nhóm nên ta **có thể tiến hành kiểm định ANOVA**

### TIẾN HÀNH PHÂN TÍCH ANOVA 1 CHIỀU

> **Bước 1**: Lấy dữ liệu và tính toán các thông số cần thiết

*   Dữ liệu được sử dụng là sample_converted_by_df (Dữ liệu đã được xử lí từ owan01.csv)
*   alpha = 0.05 là ý nghĩa của bài toán trong quá trình kiểm định

> **Bước 2**: Phát biểu giả thuyết

Giả thuyết $H_0$:

$$H_0: \text{Không có sự khác biệt về trung bình độ sâu của các nhóm}$$

Giả thuyết $H_a$:

$$H_a: \text{Có sự khác biệt về trung bình độ sâu của các nhóm}$$

> **Bước 3**: Tiến hành kiểm định ANOVA 1 chiều

In [10]:
# Khởi tạo 2 biến f và p dùng để lưu trữ giá trị f stat và p value
f, p = st.f_oneway(sample_converted_by_df[0], sample_converted_by_df[1], sample_converted_by_df[2], sample_converted_by_df[3], sample_converted_by_df[4], sample_converted_by_df[5], sample_converted_by_df[6])

# Xuất giá trị p ra màn hình
print(f"Giá trị p khi tiến hành kiểm định: {np.round(p, 10)}")

Giá trị p khi tiến hành kiểm định: 0.0


> **Bước 4**: Kết luận dựa trên kết quả kiểm định

In [11]:
if (p < alpha):
  print(f"Vì giá trị p  bé hơn mức ý nghĩa của bài toán ({np.round(p, 10)} < {alpha}). Vì vậy không có bằng chứng bác bỏ giả thuyết H0")
else:
  print(f"Vì giá trị p không bé hơn mức ý nghĩa của bài toán ({np.round(p, 10)} >= {alpha}). Vì vậy có bằng chứng bác bỏ giả thuyết H0")

Vì giá trị p  bé hơn mức ý nghĩa của bài toán (0.0 < 0.05). Vì vậy không có bằng chứng bác bỏ giả thuyết H0
